In [ ]:
# !pip install pyspark azure-storage-blob


In [ ]:
from azure.storage.blob import BlobServiceClient

# Azure Blob Storage connection string
azure_blob_connection_string = "cNvsvKyR7q7rnqcpX1nU32d65m71zwW4AQ6UM5jmnDem/20inTNMBW0LnGan3xV4xTqaUiNfD2Olgr8EkJ12QMmOcFh3zU2UsI6Ylocjzd2xI4+RDGr8g6WlsfviHWD+5M70kE9pC0Gn0TMt4gWhn9pWBrL+TVgXr3HdfAeikCQW2FMuFPtBWphBWgiqp6VQsW9HvObAgYrEN8x+4tFlxc1vD2JsCHn74eqhhh05vzC7Inodp6+R6uzlOqAuJV2B"

# Local file path to upload (use raw string or escape backslashes)
local_file_path = r"D:\Downloads 2\George Brown\Sem 2.5\Big data 2\Big Data project\Files to Submit\Transformations\Hadoop_MVCdataNYPD\hadoop_joined_crash_vehicles.csv"

import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(local_file_path)

# Get the column names from the DataFrame
column_names = df.columns.tolist()

# Print the column names
print("Column names in the CSV file:")
print(column_names)

# Blob Storage container name and blob name
container_name = "hadoopcrashvehicles"  # Replace with your container name
blob_name = "hadoop_joined_crash_vehicles.csv"  # Replace with the name you want to give to your blob/file

try:
    # Initialize BlobServiceClient using connection string
    blob_service_client = BlobServiceClient.from_connection_string(azure_blob_connection_string)

    # Create a blob client using the container name and blob name
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

    # Upload a file to Azure Blob Storage
    with open(local_file_path, "rb") as data:
        blob_client.upload_blob(data)

    print(f"File '{blob_name}' uploaded to Azure Blob Storage successfully.")

except Exception as ex:
    print(f"Error uploading file to Azure Blob Storage: {ex}")


C:\Users\macwa\AppData\Local\Temp\ipykernel_32220\1852528703.py:12: DtypeWarning: Columns (33,36,37,39,41,42,43,44,45,46,47,48,49,50,51,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(local_file_path)


Column names in the CSV file:
['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME', 'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5', 'UNIQUE_ID', 'VEH_COLLISION_ID', 'CRASH_DATE', 'CRASH_TIME', 'VEHICLE_ID', 'STATE_REGISTRATION', 'VEHICLE_TYPE', 'VEHICLE_MAKE', 'VEHICLE_MODEL', 'VEHICLE_YEAR', 'TRAVEL_DIRECTION', 'VEHICLE_OCCUPANTS', 'DRIVER_SEX', 'DRIVER_LICENSE_STATUS', 'DRIVER_LICENSE_JURISDICTION', 'PRE_CRASH', 'PO

In [2]:
import pyodbc

# Azure SQL Database connection details
server = 'blahblah.database.windows.net'
database = 'Hadoop_CrashVehiclesDataNYPD'
username = 'azureuser'
password = 'Ilove85workWonder69'
driver = '{ODBC Driver 18 for SQL Server}'

# Create the connection string
conn_str = (
    f'DRIVER={driver};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password};'
    f'Encrypt=yes;'
    f'TrustServerCertificate=no;'
    f'Connection Timeout=30;'
)
# Adjust the data types according to your actual data
columns_with_types = ", ".join([f"[{col}] VARCHAR(255)" for col in column_names])

# Define the CREATE TABLE statement
table_name = "hadoop_joined_crash_vehicles"
create_table_sql = f"CREATE TABLE {table_name} ({columns_with_types});"

try:
    # Establish a connection to the Azure SQL Database
    connection = pyodbc.connect(conn_str)
    
    # Print connection details
    print("Connection established successfully!")
    print(connection.getinfo(pyodbc.SQL_DBMS_NAME))
    print(connection.getinfo(pyodbc.SQL_DBMS_VER))

    # Execute the CREATE TABLE statement
    cursor = connection.cursor()
    cursor.execute(create_table_sql)
    connection.commit()

except pyodbc.Error as ex:
    print("Error connecting to Azure SQL Database:", ex)

Connection established successfully!
Microsoft SQL Server
12.00.5564


In [ ]:
def log_message(message):
    """Function to log general messages"""
    with open('process_log.txt', 'a') as f:
        f.write(message + '\n')

def log_error(error_message):
    """Function to log errors"""
    with open('error_log.txt', 'a') as f:
        f.write(error_message + '\n')

try:
    log_message("Attempting to establish connection to Azure SQL Database...")
    # Establish a connection to the Azure SQL Database
    connection = pyodbc.connect(conn_str)
    log_message("Connection established successfully!")

    # Create a cursor object
    cursor = connection.cursor()

    log_message("Reading CSV file into pandas DataFrame...")
    # Read the CSV file into a pandas DataFrame
    local_file_path = r"D:\Downloads 2\George Brown\Sem 2.5\Big data 2\Big Data project\Files to Submit\Transformations\Hadoop_MVCdataNYPD\hadoop_joined_crash_vehicles.csv"
    df = pd.read_csv(local_file_path)
    log_message("CSV file read successfully!")

    # Convert data types if necessary
    df = df.astype(str)
    log_message("Converted DataFrame columns to strings.")

    # Print the DataFrame structure
    log_message(f"DataFrame structure: \n{df.info()}")

    # Use the entire DataFrame for insertion
    df_subset = df
    log_message(f"Using all rows of the DataFrame.")

    # Print the first few rows
    log_message(f"First few rows of the DataFrame subset:\n{df_subset.head()}")

    # Insert DataFrame records one by one, commit in batches
    batch_size = 1000  # Adjust the batch size as needed
    batch_values = []

    for index, row in df_subset.iterrows():
        values = tuple(row)
        batch_values.append(values)

        if (index + 1) % batch_size == 0 or (index + 1) == len(df_subset):
            try:
                insert_sql = f"INSERT INTO hadoop_joined_crash_vehicles VALUES ({', '.join('?' * len(values))})"
                cursor.executemany(insert_sql, batch_values)
                connection.commit()
                log_message(f"Committed {index + 1} rows successfully.")
                batch_values.clear()
            except pyodbc.Error as e:
                error_message = f"Error inserting batch ending at row {index + 1}: {str(e)}"
                log_error(error_message)
                log_message(error_message)
                connection.rollback()  # Rollback the batch on error

    # Final commit for any remaining rows
    if batch_values:
        try:
            cursor.executemany(insert_sql, batch_values)
            connection.commit()
            log_message(f"Committed final batch successfully.")
        except pyodbc.Error as e:
            error_message = f"Error inserting final batch: {str(e)}"
            log_error(error_message)
            log_message(error_message)
            connection.rollback()  # Rollback the final batch on error

    log_message("Data inserted successfully into the 'call_data_lapd' table.")

    # Close the cursor and connection
    cursor.close()
    connection.close()
    log_message("Connection closed successfully.")

except pyodbc.Error as ex:
    # Log the connection error to the file
    error_message = f"Error connecting to Azure SQL Database: {str(ex)}"
    log_error(error_message)
    log_message(error_message)

except Exception as ex:
    # Log any other exceptions
    error_message = f"General error: {str(ex)}"
    log_error(error_message)
    log_message(error_message)

C:\Users\macwa\AppData\Local\Temp\ipykernel_32220\2749139235.py:23: DtypeWarning: Columns (33,36,37,39,41,42,43,44,45,46,47,48,49,50,51,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(local_file_path)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4219296 entries, 0 to 4219295
Data columns (total 54 columns):
 #   Column                         Dtype 
---  ------                         ----- 
 0   CRASH DATE                     object
 1   CRASH TIME                     object
 2   BOROUGH                        object
 3   ZIP CODE                       object
 4   LATITUDE                       object
 5   LONGITUDE                      object
 6   LOCATION                       object
 7   ON STREET NAME                 object
 8   CROSS STREET NAME              object
 9   OFF STREET NAME                object
 10  NUMBER OF PERSONS INJURED      object
 11  NUMBER OF PERSONS KILLED       object
 12  NUMBER OF PEDESTRIANS INJURED  object
 13  NUMBER OF PEDESTRIANS KILLED   object
 14  NUMBER OF CYCLIST INJURED      object
 15  NUMBER OF CYCLIST KILLED       object
 16  NUMBER OF MOTORIST INJURED     object
 17  NUMBER OF MOTORIST KILLED      object
 18  CONTRIBUTING FACTOR VE